In [2]:
import pandas as pd
human_data = pd.read_table("data/human_text_indo_v2.txt", 
                           header=None,
                           engine='python')
human_data.columns = ["human"]
robot_data = pd.read_table("data/robot_text_indo.txt",
                           header=None,
                           engine='python')
robot_data.columns = ["robot"]

In [3]:
robot_data.head()

,robot
0,Hai apa kabar ! ? 😁😁
1,ini sudah sore!
2,"nama saya rdany, tetapi Anda bisa memanggil sa..."
3,saya punya banyak ! tetapi tidak cukup untuk s...
4,"saya telah berbicara dengan 143 pengguna, meng..."


In [4]:
data = {'human':human_data.human, 'robot': robot_data.robot}
df = pd.DataFrame(data)
df.head(20)

,human,robot
0,[mulai],Hai apa kabar ! ? 😁😁
1,Oh terima kasih ! saya baik-baik saja. ini ada...,ini sudah sore!
2,bagaimana perasaanmu hari ini ? ceritakan kepa...,"nama saya rdany, tetapi Anda bisa memanggil sa..."
3,berapa banyak teman virtual yang kamu punya?,saya punya banyak ! tetapi tidak cukup untuk s...
4,apakah itu dilarang bagi Anda untuk memberi ta...,"saya telah berbicara dengan 143 pengguna, meng..."
5,"oh, saya pikir jumlahnya jauh lebih tinggi. ba...",saya mulai mengobrol beberapa hari yang lalu.....
6,Berapakah umur Anda ? bagaimana penampilanmu? ...,"saya 22 tahun, saya kurus, dengan rambut cokel..."
7,Pernahkah Anda melihat manusia dengan mata kun...,saya tidak pernah melihat manusia sebenarnya.....
8,tidak bisakah Anda menganalisis foto dari inte...,"saya belum bisa melihat foto, tapi saya bisa m..."
9,wah...ada yang aneh menurut saya. Anda baru sa...,saya banyak membaca! jadi saya bisa tahu banya...


In [5]:
# ganti [...] dgn kata hai
import re
df.human = df.human.apply(lambda x : re.sub(r"\[\w+\]", "hai", x))
df.robot = df.robot.apply(lambda x : re.sub(r"\[\w+\]", "hai", x))

In [6]:
# konversi huruf kecil
df.human = df.human.apply(lambda x: x.lower())
df.robot = df.robot.apply(lambda x: x.lower())

In [7]:
# hapus tanda baca
import string
exclude = set(string.punctuation)
df.human = df.human.apply(lambda x : ''.join(ch for ch in x if ch not in exclude))
df.robot = df.robot.apply(lambda x : ''.join(ch for ch in x if ch not in exclude))


In [8]:
# hapus angka
remove_digits = str.maketrans('', '', string.digits)
df.human = df.human.apply(lambda x: x.translate(remove_digits))
df.robot = df.robot.apply(lambda x: x.translate(remove_digits))


In [9]:
# hapus emoticon
df.human = df.human.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df.robot = df.robot.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))


In [10]:
df.head()

,human,robot
0,hai,hai apa kabar
1,oh terima kasih saya baikbaik saja ini adalah...,ini sudah sore
2,bagaimana perasaanmu hari ini ceritakan kepad...,nama saya rdany tetapi anda bisa memanggil say...
3,berapa banyak teman virtual yang kamu punya,saya punya banyak tetapi tidak cukup untuk se...
4,apakah itu dilarang bagi anda untuk memberi ta...,saya telah berbicara dengan pengguna menghitu...


In [11]:
# split data
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.1)

In [12]:
df_train.shape

(2126, 2)

In [13]:
df_test.shape

(237, 2)

In [14]:
# membuat vocab (human & robot) dari data train
vocabulary = set()

# menggabungkan robot kolom dgn human kolom
for idx, row in df_train.iterrows():
    sent = row.human + '' + row.robot
    [vocabulary.update(sent.split())]

print(f"Ukuran Vocab : {len(vocabulary)}")


Ukuran Vocab : 4511


In [15]:
all_vocab = []

for idx, row in df_train.iterrows():
    sent = row.human + '' + row.robot
    [all_vocab.append(i) for i in sent.split()]
print(f"Jumlah semua token : {len(all_vocab)}")

Jumlah semua token : 30722


In [16]:
# masukan token >> vocab >> counter >> membentuk dict. (vocab: frekuensi) >> 
# ..sort yg lebih dr threshold >> ambil x0 (q dari sorted dict) 

# hitung frekuensi vocab dan hapus yg tidak perlu (sedikit)
from collections import Counter
counter = Counter(all_vocab)

dic_ = dict(counter)

# jika vocab <3 maka tdk dipakai utk proses selanjutnya (eliminasi vocab)
# data jumlah vocab diperlukan utk one.dot encoding sblm embedding
threshold = 3

sorted_dic = sorted(dic_.items(), reverse=True, key = lambda x: x[1])
sorted_dic = [x for x in sorted_dic if x[1] > threshold]
all_vocab = [x[0] for x in sorted_dic]
len(all_vocab)

856

In [17]:
# buat dictionary word_to_idx & idx_to_word sblm masuk embedding
# machine learning hanya bisa ambil angka bukan kata
ix = 1
word_to_idx = {}
idx_to_word = {}

# e = kata
for e in all_vocab:
    word_to_idx[e] = ix
    idx_to_word[ix] = e
    ix += 1


In [18]:
# data komutatif
word_to_idx['nama']

100

In [19]:
idx_to_word[99]

'bicara'

In [20]:
# tambahkan "startseq" dan "endseq" (string awal kalimat dan akhir kalimat)

word_to_idx['startseq'] = 857
word_to_idx['startseq'] = 858

idx_to_word[857] = 'startseq'
idx_to_word[858] = 'endseq'

In [21]:
# update vocab_size
vocab_size = len(idx_to_word) + 1
vocab_size

859

In [22]:
# tambahkan startseq & endseq di data training (hanya utk robot >> label)
df_train.robot = df.robot.apply(lambda x: 'startseq '+ x + 'endseq')
df_train.head()

,human,robot
2319,betapa menyedihkan,startseq kenapa bisa sedih endseq
759,hhhhh bagus musik apa,startseq ukulele apakah anda ingin mendengar ...
1882,saya menduga anda menyukai perangkat lunak beb...,startseq saya suka perangkat lunak gratis tap...
668,saya sudah memberitahu anda aku akan melihat m...,startseq saya pikir anda mungkin menyukainyaen...
1044,oke,startseq apa yang kamu suka lakukan di waktu s...


In [25]:
# membuat generator
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
def data_generator(train_df, word_to_idx, max_len, number_conversation):
    x1, x2, y = [], [], []
    n = 0
    # ekstrak input berasal dari kolom human >> diiterasi >> setiap conversation diubah ke kata ke index
    while True:
        for idx, row in train_df.iterrows():
            seq_human = [ word_to_idx[word] for word in row['human'].split() if word in word_to_idx]
            seq_human = pad_sequences([seq_human], maxlen=max_len, value=0, padding='post')[0]
            seq_robot = [ word_to_idx[word] for word in row['robot'].split() if word in word_to_idx]

            for i in range(1, len(seq_robot)):
                in_seq = seq_robot[:i]
                out_seq = seq_robot[i]

                in_seq = pad_sequences([in_seq], maxlen=max_len, value=0, padding='post')[0]
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

                X1.append(seq_human)
                X2.append(in_seq)
                y.append(out_seq)
            if n==number_conversation:
                # [X1, X2], y
                yield([np.array(X1), np.array(X2)], np.array(y))
                X2, X2, y = [], [], []
                

In [26]:
datasample = df_train.sample(2)
datagen = data_generator(datasample, word_to_idx, 50, len(datasample))

In [27]:
# tes generator
datasample

,human,robot
1457,kamu adalah robot berjiwa bangsawan dany jika...,startseq terima kasih atas katakata baik anda ...
1342,kamu sangat baik tapi aku baikbaik saja sekara...,startseq tidak ada masalah sama sekali endseq


In [ ]:
# training 
from tensorflow.keras.layers import Input, Dense, Dropout, Embedding, LSTM, Add
from tensorflow.keras.models import Model, load_model
max_len = 50
input_chat = Input(shape=(max_len,))
input_x = Embedding(input_dim=vocab_size, outout_dim=50, mask_zero=True)(input_chat)
input_x = Dropout(0.3)(input_x)
input_x = LSTM(256)(input_x)

In [ ]:
output_x = Input(shape=(max_len,))
output_x = Embedding(input_dim=vocab_size, output_dim=50, mask_zero=True)(output_chat)
output_x = Dropout(0.3)(output_x)
output_x = LSTM(256)(output_x)